<a href="https://colab.research.google.com/github/Naeem-Ul/Naeem-Ul.github.io/blob/master/Ensembleall_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "tensorflow-text==2.8.*"
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
!pip install transformers
#loading data
import numpy as np
import pandas as pd
df = pd.read_csv('augmented_dementiabank.csv',encoding ='latin-1',names=('target','text'))
df.head(5)
df['target'].value_counts()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['target'], test_size=0.2, random_state=0)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.1/498.1 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 53.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
np.savez('X_train.npz',X_train)
np.savez('X_test.npz',X_test)
np.savez('y_train.npz',y_train)
np.savez('y_test.npz',y_test)
X_train=np.load('X_train.npz',allow_pickle=True)
print(X_train['arr_0'].shape)
print(X_train['arr_0'])
X_train = np.load('X_train.npz', allow_pickle=True)['arr_0']
y_train = np.load('y_train.npz', allow_pickle=True)['arr_0']
X_test = np.load('X_test.npz', allow_pickle=True)['arr_0']
y_test = np.load('y_test.npz', allow_pickle=True)['arr_0']

(1195,)
[" well te kd the girl's laughing at her brother because he went into the cookie jar nd he's falling over the cookie jar . and mother's the mother was t the sink . and the sink's splashing splashing over the sink . and she sortof a little bt bitchy . and the water' going flooding over the sink . and there's a saucer, there's a plate, there's a couple of dishes . ell she's and the mother's looking out the window . she don't knw hat the hell to think of it . a girl laughing at her rother who is taking cookies outof the cokie jar and he's ready to fall off the damn off the off the chair he's on . the chair's crooked . what the hell else ? nd he there's a plate, saucer and two cups . she's looking out the window . the window's open . she's not paying any attention o the kids . the water's flooding over the sink . what else you want ? "
 " oh I see the sink is running over . and I see the stool is tipping ove . the little boy i trying to get cookies out . the little girl is  reachin

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
## Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [ ]:
bert_outputs = bert_encoder(preprocessed_text)

# Convolutional layer
conv_layer = tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(bert_outputs['sequence_output'])

# Recurrent layer
recurrent_layer = tf.keras.layers.LSTM(64, return_sequences=True)(conv_layer)

# Pooling layer
pooling_layer = tf.keras.layers.GlobalMaxPooling1D()(recurrent_layer)

# Dense layers
dense_layer = tf.keras.layers.Dense(128, activation='relu')(pooling_layer)
dense_layer = tf.keras.layers.Dense(128, activation='relu')(pooling_layer)
dense_layer = tf.keras.layers.Dense(128, activation='relu')(pooling_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(dense_layer)

# Construct the final model
model = tf.keras.Model(inputs=text_input, outputs=output_layer)

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer,
              loss='binary_crossentropy',
              metrics=METRICS)
checkpoint_path = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
cp = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1, mode='max')
csv_logger = tf.keras.callbacks.CSVLogger('logfile1.log')
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test,y_test),callbacks=[cp, csv_logger,reduce_lr])

Epoch 1/50
38/38 [==============================] - ETA: 0s - loss: 0.5861 - accuracy: 0.6854 - precision: 0.6667 - recall: 0.7772
Epoch 1: val_accuracy improved from -inf to 0.77258, saving model to weights-improvement-01-0.77.hdf5
38/38 [==============================] - 48s 559ms/step - loss: 0.5861 - accuracy: 0.6854 - precision: 0.6667 - recall: 0.7772 - val_loss: 0.4966 - val_accuracy: 0.7726 - val_precision: 0.8761 - val_recall: 0.6471 - lr: 0.0010
Epoch 2/50
38/38 [==============================] - ETA: 0s - loss: 0.3982 - accuracy: 0.8243 - precision: 0.8610 - recall: 0.7854
Epoch 2: val_accuracy did not improve from 0.77258
38/38 [==============================] - 17s 451ms/step - loss: 0.3982 - accuracy: 0.8243 - precision: 0.8610 - recall: 0.7854 - val_loss: 0.6194 - val_accuracy: 0.6355 - val_precision: 0.5894 - val_recall: 0.9477 - lr: 0.0010
Epoch 3/50
38/38 [==============================] - ETA: 0s - loss: 0.3476 - accuracy: 0.8510 - precision: 0.8600 - recall: 0.8488


In [ ]:
## Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(64, activation='relu')(l)
#l = tf.keras.layers.Flatten()(l)  # Use tf.keras.layers.Flatten() instead of Flatten()
#l = tf.keras.layers.Dense(3, activation='softmax')(l)  # Add missing Dense layer
l = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(l)
l = tf.keras.layers.Dense(128, activation='relu')(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model2 = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer,
              loss='binary_crossentropy',
              metrics=METRICS)
checkpoint_path = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
cp = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1, mode='max')
csv_logger = tf.keras.callbacks.CSVLogger('logfile2.log')
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2)
model2.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test,y_test),callbacks=[cp, csv_logger,reduce_lr])

Epoch 1/50
38/38 [==============================] - ETA: 0s - loss: 1.3555 - accuracy: 0.5146 - precision: 0.5271 - recall: 0.5528
Epoch 1: val_accuracy improved from -inf to 0.55518, saving model to weights-improvement-01-0.56.hdf5
38/38 [==============================] - 23s 532ms/step - loss: 1.3555 - accuracy: 0.5146 - precision: 0.5271 - recall: 0.5528 - val_loss: 1.1598 - val_accuracy: 0.5552 - val_precision: 0.5368 - val_recall: 0.9542 - lr: 0.0010
Epoch 2/50
38/38 [==============================] - ETA: 0s - loss: 1.0414 - accuracy: 0.6008 - precision: 0.6142 - recall: 0.6033
Epoch 2: val_accuracy improved from 0.55518 to 0.65217, saving model to weights-improvement-02-0.65.hdf5
38/38 [==============================] - 19s 504ms/step - loss: 1.0414 - accuracy: 0.6008 - precision: 0.6142 - recall: 0.6033 - val_loss: 0.9077 - val_accuracy: 0.6522 - val_precision: 0.7290 - val_recall: 0.5098 - lr: 0.0010
Epoch 3/50
38/38 [==============================] - ETA: 0s - loss: 0.8643 - 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
# Obtain predictions from Model 1 and Model 2
predictions_model = model.predict(X_test)
predictions_model2 = model2.predict(X_test)

# Perform averaging
ensemble_predictions = np.round((predictions_model + predictions_model2) / 2)

# Evaluate the performance of the ensemble
accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {accuracy}")


Ensemble Accuracy: 0.9498327759197325


In [ ]:
# Sample data
sample_dataset = [
" the scene is in the in the kitchen . the mother is wiping dishes and the water is running on the floor . a child is trying to get a boy is trying to get cookies outta out a jar and he's about to tip over on a stool . the little girl is reacting to his falling . it seems to be summer out . the window is open . the curtains are blowing . it must be a gentle breeze . there's grass outside in the garden . mother's finished certain of the the dishes . kitchen's very tidy . the mother seems to have nothing in the house to eat except cookies in the cookie jar . the children look to be almost about the same size . perhaps they're twins . they're dressed for summer warm weather . you want more ? the mother's in a short sleeve dress . I'll hafta say it's warm . " ,
                   "  it's kinda shiny there . I   okay it looks like somebody's raiding the cookie jar .  there's a woman  working in the kitchen .  and there's this little girl here with something but I don't know what it is she yeah I'm trying to identify this thing . d?d?ku trying to see . is that a table leg ?  I'd say that's a table leg . ",
                  "the little boy is reaching for a cookie . and he's falling at the same time because the stool has tipped over . his little sister is reaching for a cookie and I think beginning to laugh because he's falling . the sink is running over and dripping water on the floor while the mother is trying to dry the dishes . the window is open and the curtains are blowing in the breeze coming in from outside . the water is running in the sink . that's why it's overflowing .  I also believe that the mother is daydreaming .  possibly worrying about something . that's it . that's all I see . " ]

# Make predictions using the ensemble model
ensemble_predictions = np.round((model.predict(sample_dataset) + model2.predict(sample_dataset)) / 2)

# Display the predictions
print("Ensemble Predictions:")
for prediction in ensemble_predictions:
    print(prediction)


Ensemble Predictions:
[0.]
[1.]
[0.]


In [ ]:
predictions_model = model.predict(X_test)
predictions_model2 = model2.predict(X_test)

In [ ]:
y_predicted_model = model.predict(X_test)
y_predicted_model2 = model2.predict(X_test)
y_predicted_model = y_predicted_model.flatten()
y_predicted_model2 = y_predicted_model2.flatten()

In [ ]:
sample_dataset = [
" the scene is in the in the kitchen . the mother is wiping dishes and the water is running on the floor . a child is trying to get a boy is trying to get cookies outta out a jar and he's about to tip over on a stool . the little girl is reacting to his falling . it seems to be summer out . the window is open . the curtains are blowing . it must be a gentle breeze . there's grass outside in the garden . mother's finished certain of the the dishes . kitchen's very tidy . the mother seems to have nothing in the house to eat except cookies in the cookie jar . the children look to be almost about the same size . perhaps they're twins . they're dressed for summer warm weather . you want more ? the mother's in a short sleeve dress . I'll hafta say it's warm . " ,
                   "  it's kinda shiny there . I   okay it looks like somebody's raiding the cookie jar .  there's a woman  working in the kitchen .  and there's this little girl here with something but I don't know what it is she yeah I'm trying to identify this thing . d?d?ku trying to see . is that a table leg ?  I'd say that's a table leg . ",
                  "the little boy is reaching for a cookie . and he's falling at the same time because the stool has tipped over . his little sister is reaching for a cookie and I think beginning to laugh because he's falling . the sink is running over and dripping water on the floor while the mother is trying to dry the dishes . the window is open and the curtains are blowing in the breeze coming in from outside . the water is running in the sink . that's why it's overflowing .  I also believe that the mother is daydreaming .  possibly worrying about something . that's it . that's all I see . " ]
model.predict(sample_dataset)

array([[0.11824717],
       [0.501327  ],
       [0.06559467]], dtype=float32)

In [ ]:
sample_dataset = [
" the scene is in the in the kitchen . the mother is wiping dishes and the water is running on the floor . a child is trying to get a boy is trying to get cookies outta out a jar and he's about to tip over on a stool . the little girl is reacting to his falling . it seems to be summer out . the window is open . the curtains are blowing . it must be a gentle breeze . there's grass outside in the garden . mother's finished certain of the the dishes . kitchen's very tidy . the mother seems to have nothing in the house to eat except cookies in the cookie jar . the children look to be almost about the same size . perhaps they're twins . they're dressed for summer warm weather . you want more ? the mother's in a short sleeve dress . I'll hafta say it's warm . " ,
                   "  it's kinda shiny there . I   okay it looks like somebody's raiding the cookie jar .  there's a woman  working in the kitchen .  and there's this little girl here with something but I don't know what it is she yeah I'm trying to identify this thing . d?d?ku trying to see . is that a table leg ?  I'd say that's a table leg . ",
                  "the little boy is reaching for a cookie . and he's falling at the same time because the stool has tipped over . his little sister is reaching for a cookie and I think beginning to laugh because he's falling . the sink is running over and dripping water on the floor while the mother is trying to dry the dishes . the window is open and the curtains are blowing in the breeze coming in from outside . the water is running in the sink . that's why it's overflowing .  I also believe that the mother is daydreaming .  possibly worrying about something . that's it . that's all I see . " ]
model2.predict(sample_dataset)

array([[0.11824717],
       [0.501327  ],
       [0.06559467]], dtype=float32)